In [2]:
# | default_exp routes.pdp

In [3]:
# | export
import httpx
import pandas as pd
import io

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

In [4]:
# | export
class PDP_NotRetrieved(de.DomoError):
    def __init__(
        self,
        domo_instance,
        function_name,
        status,
        message,
        pdp_id=None,
    ):

        super().__init__(
            domo_instance=domo_instance,
            entity_id=pdp_id,
            function_name=function_name,
            status=status,
            message=message,
        )

In [5]:
# | export
async def get_pdp_policies(
    auth: dmda.DomoAuth,
    dataset_id: str,
    debug_api: bool = False,
) -> rgd.ResponseGetData:
    url = f"http://{auth.domo_instance}.domo.com/api/query/v1/data-control/{dataset_id}/filter-groups/"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth, 
        url=url, 
        method="GET", 
        debug_api=debug_api, 
        # headers= {'accept': 'application/json'},
        # params = {'options':'load_associations,load_filters,include_open_policy'},
        is_follow_redirects=True
    )

    if len(res.response) == 0 or not res.is_success:
        raise PDP_NotRetrieved(
            domo_instance=auth.domo_instance,
            function_name="get_pdp_policies",
            status=res.status,
            message="failed to retrieve pdp policies",
        )

    return res

### Sample implementation for get_pdp_policies

In [6]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

dataset_id = os.environ["DOJO_DATASET_ID"]

try:
    res = await get_pdp_policies(auth=token_auth, dataset_id=dataset_id)
    print(res)
except PDP_NotRetrieved as e:
    print(e)


ResponseGetData(status=200, response=[{'name': 'Test Policy Name Employee 123', 'filterGroupId': 1153, 'dataSourceId': '0e7f7755-eccc-4612-890c-c3dc3d5002c3', 'type': 'user', 'dataSourcePermissions': False, 'parameters': [{'name': 'Employee ID', 'value': '456', 'values': ['456'], 'type': 'COLUMN', 'operator': 'EQUALS', 'not': False, 'ignoreCase': True}]}, {'name': 'Test 456', 'filterGroupId': 1156, 'dataSourceId': '0e7f7755-eccc-4612-890c-c3dc3d5002c3', 'type': 'user', 'userIds': [2072616249], 'dataSourcePermissions': False, 'resources': {'USER': ['2072616249']}, 'parameters': [{'name': 'Employee ID', 'value': '465', 'values': ['465'], 'type': 'COLUMN', 'operator': 'EQUALS', 'not': False, 'ignoreCase': False}]}, {'name': 'Test 94', 'filterGroupId': 1157, 'dataSourceId': '0e7f7755-eccc-4612-890c-c3dc3d5002c3', 'type': 'user', 'userIds': [1893952720], 'dataSourcePermissions': False, 'resources': {'USER': ['1893952720']}, 'parameters': [{'name': 'Employee ID', 'value': '465', 'values': ['

In [7]:

# class SearchPDP_Error(de.DomoError):
#     def __init__(self, status, message, domo_instance, function_name = "search_pdp_by_name"):
#         super().__init__(function_name = function_name, status = status, message = message , domo_instance = domo_instance)
        
# def search_pdp_policies_by_name(
#         # used to return pdp policy info, search by name
#         search_name: str,
#         result_list: list[dict], # this is the res.response from get_pdp_policies -- should be list of dict
#         is_exact_match: bool = True
#         ):
    
#     if is_exact_match:
#         policy_search = next((policy for policy in result_list if policy.name == search_name), None)
#         #print(policy_search)   
         
#         if not policy_search:
#             raise SearchPDP_Error(
#                 status='',
#                 message=f'There is no policy named "{search_name}"',
#                 domo_instance=''
#             )  
          
#         return policy_search
#     else:
#         policy_search = [policy for policy in result_list if search_name.lower() in policy.name.lower()]
#         if not policy_search:
#             raise SearchPDP_Error(
#                 status='',
#                 message=f'There is no policy name containing "{search_name}"',
#                 domo_instance=''
#             )  
        
#         return policy_search

In [8]:
# pdp_policy_search = search_pdp_policies_by_name(search_name='1', result_list=res.response, is_exact_match=False)
# pdp_policy_search

AttributeError: 'dict' object has no attribute 'name'

Bad pipe message: %s [b"sj\x93\xbd\xd9\xeeF\xfdr\xa9Q!Y\xe6\xe0G\x97\x0f\x00\x00\xf4\xc00\xc0,\xc0(\xc0$\xc0\x14\xc0\n\x00\xa5\x00\xa3\x00\xa1\x00\x9f\x00k\x00j\x00i\x00h\x009\x008\x007\x006\x00\x88\x00\x87\x00\x86\x00\x85\xc0\x19\x00\xa7\x00m\x00:\x00\x89\xc02\xc0.\xc0*\xc0&\xc0\x0f\xc0\x05\x00\x9d\x00=\x005\x00\x84\xc0/\xc0+\xc0'\xc0#\xc0\x13\xc0\t\x00\xa4\x00\xa2\x00\xa0\x00\x9e\x00g\x00@\x00?\x00>\x003\x002\x001\x000\x00\x9a\x00\x99\x00\x98\x00\x97\x00E\x00D\x00C\x00B\xc0\x18\x00\xa6\x00l\x004\x00\x9b\x00F\xc01\xc0-\xc0)\xc0%\xc0\x0e\xc0\x04\x00\x9c\x00<\x00/\x00\x96\x00A\x00\x07\xc0\x11\xc0\x07\xc0\x16\x00\x18\xc0\x0c\xc0\x02\x00\x05\x00\x04\xc0\x12\xc0\x08\x00\x16\x00\x13\x00\x10\x00\r\xc0\x17\x00\x1b\xc0\r\xc0\x03\x00\n\x00\x15\x00\x12\x00\x0f\x00\x0c\x00\x1a\x00\t\x00\x14\x00\x11\x00\x19\x00\x08\x00\x06\x00\x17\x00\x03\xc0"]
Bad pipe message: %s [b'\xca\xc4f\x8aK\xeb\x9bJ\xf62\x86\x0b^\xc8\xba\xc8\xb8\xaf D\xb6w?\x85\x8d\xd1,Pz\xb6\xb7\xbeg\xab\xcf\xfe\xe9oD\xfdI\xd3\xb4\x17!\x

In [ ]:

# class SearchPDP_Error(de.DomoError):
#     def __init__(self, status, message, domo_instance, function_name = "search_pdp_by_name"):
#         super().__init__(function_name = function_name, status = status, message = message , domo_instance = domo_instance)

# async def search_pdp_policies_by_id(
#         # used to return pdp policy info, search by id
#         search_id: str,
#         result_list: list[dict], # this is the res.response from get_pdp_policies -- should be list of dict
#         ):
    
#     policy_search = next((policy for policy in result_list if policy.filter_group_id == search_id), None)
#     #print(policy_search)   
         
#     if not policy_search:
#         raise SearchPDP_Error(
#             status='',
#             message=f'There is no policy id "{search_id}"',
#             domo_instance=''
#         )  
          
#     return policy_search


## generate body for create_policy endpoint


In [ ]:
# | export
def generate_policy_parameter_simple(
    column_name: str,
    column_values_ls: list[str] = None,
    operator="EQUALS",
    ignore_case: bool = True,
):
    if not isinstance(column_values_ls, list):
        column_values_ls = [column_values_ls]

    return {
        "type": "COLUMN",
        "name": column_name,
        "values": column_values_ls,
        "operator": operator,
        "ignoreCase": ignore_case,
    }

In [ ]:
# | export
def generate_policy_body(
    policy_name: str,
    dataset_id: str,
    parameters_ls: list[dict],  # generated by generate_policy_parameter_simple method
    policy_id: str = None,  # only included if updating existing policy
    user_ids: list[str] = None,
    group_ids: list[str] = None,
    virtual_user_ids: list[str] = None,
):
    if not user_ids:
        user_ids = []

    if not group_ids:
        group_ids = []

    if not virtual_user_ids:
        virtual_user_ids = []

    if not isinstance(parameters_ls, list):
        parameters_ls = [parameters_ls]


    body = {
        "name": policy_name,
        "dataSourceId": dataset_id,
        "userIds": user_ids,
        "virtualUserIds": virtual_user_ids,
        "groupIds": group_ids,
        "dataSourcePermissions": False,
        "parameters": parameters_ls,
    }

    if policy_id:
        body.update({"filterGroupId": policy_id})

    return body

## Policy CRUD Operations

In [ ]:
# | export
class CreatePolicy_Error(de.DomoError):
    def __init__(self, status, message, domo_instance, function_name = "create_policy"):
        super().__init__(function_name = function_name, status = status, message = message , domo_instance = domo_instance)

async def create_policy(
    auth: dmda.DomoAuth,
    dataset_id: str,
    body: dict,  # generated using generate_policy_parameter_simple & generate_policy_body
    session: httpx.AsyncClient = None,
    override_same_name: bool = False,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/query/v1/data-control/{dataset_id}/filter-groups"

    if debug_api:
        print(url)
    
    if override_same_name:
        print(f'Creating policy...')
        res = await gd.get_data(
            auth=auth,
            url=url,
            method="POST",
            body=body,
            debug_api=debug_api,
            session=session,
        )
    else:
        existing_policies = await get_pdp_policies(auth=auth, dataset_id=dataset_id) 
        
        if existing_policies.is_success:
            try:
                policy_exists = search_pdp_policies_by_name(search_name=body.get('name'), result_list=existing_policies.response, is_exact_match=True)
            except:
                policy_exists = False
            
            if policy_exists:
                raise CreatePolicy_Error(
                    status='',
                    message= f'Policy name already exists--avoid creating pdp policies with the same name..To override, rerun and set "override_same_name=True"',
                    domo_instance=auth.domo_instance,
                )
            else:
                print(f'Creating policy...')
                res = await gd.get_data(
                auth=auth,
                url=url,
                method="POST",
                body=body,
                debug_api=debug_api,
                session=session,
            )

    return res

#### sample implementation of create_policy, generate_policy_parameter_simple and generate_policy_body

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

dataset_id = os.environ["DOJO_DATASET_ID"]

sample_parameter = generate_policy_parameter_simple(
    column_name="Employee ID",
    column_values_ls=['1234'],
    operator="EQUALS",
    ignore_case=True,
)

body = generate_policy_body(
    policy_name="Test Policy Name Employee 1234",
    dataset_id = dataset_id,
    policy_id="123",  # including the policy_id updates that existing policy
    parameters_ls=sample_parameter,
    user_ids=['1893952720']
)

res = await create_policy(
    auth = token_auth,
    dataset_id = dataset_id,
    body = body,
    override_same_name=False
)
res.response

CreatePolicy_Error: create_policy: Policy name already exists--avoid creating pdp policies with the same name..To override, rerun and set "override_same_name=True" at domo-community

In [ ]:
print(body['name'])

Test Policy Name Employee 123


In [ ]:
# | export
async def update_policy(
    auth: dmda.DomoAuth,
    dataset_id: str,
    policy_id: str,
    body: dict,  # generated using generate_policy_parameter_simple & generate_policy_body
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/query/v1/data-control/{dataset_id}/filter-groups/{policy_id}"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=body,
        debug_api=debug_api,
        session=session,
    )

    return res

#### sample implementation of update_policy

In [ ]:
import os
from pprint import pprint

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

dataset_id = os.environ["DOJO_DATASET_ID"]

sample_parameters = generate_policy_parameter_simple(
    column_name="Employee ID",
    column_values_ls=["456"],
    operator="EQUALS",
    ignore_case=True,
)

body = generate_policy_body(
    "Test Policy Name Employee 123",
    dataset_id=dataset_id,
    # policy_id='1153',
    parameters_ls=[sample_parameters],
    user_ids=['1893952720']
)

res = await update_policy(
    auth=token_auth,
    dataset_id=dataset_id,
    policy_id='1153',
    body=body,
)

res

ResponseGetData(status=200, response={'name': 'Test Policy Name Employee 123', 'filterGroupId': 1153, 'dataSourceId': '0e7f7755-eccc-4612-890c-c3dc3d5002c3', 'dataSourcePermissions': False, 'parameters': [{'name': 'Employee ID', 'value': '456', 'values': ['456'], 'type': 'COLUMN', 'operator': 'EQUALS', 'not': False, 'ignoreCase': True}]}, is_success=True)

In [ ]:
# | export

async def toggle_pdp(
        auth: dmda.DomoAuth,
        dataset_id: str,
        is_enable: bool = True,
        debug_api: bool = False,
        session: httpx.AsyncClient = None
    )-> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/query/v1/data-control/{dataset_id}"

    if debug_api:
        print(url)

    body = {"enabled": is_enable,
            "external": False  # not sure what this parameter does
            }

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=body,
        debug_api=debug_api,
        session=session,
    )

    return res


#### sample implementation of toggle_pdp

In [ ]:
import os
from pprint import pprint

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

dataset_id = os.environ["DOJO_DATASET_ID"]

res = await toggle_pdp(
    auth=token_auth,
    dataset_id=dataset_id,
    is_enable = True,
)
res

ResponseGetData(status=200, response={'enabled': True, 'secured': False, 'external': False}, is_success=True)

In [ ]:
# | hide

import nbdev

nbdev.nbdev_export()